<a href="https://colab.research.google.com/github/Abhishek-xavi/DataScienceProjects/blob/main/Text%20Preprocessing%20Methods%20for%20Deep%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connecting to Kaggle Dataset
!pip install kaggle
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
#Downloading the datset from Kaggle
!kaggle competitions download -c quora-insincere-questions-classification



100% 5.95G/5.96G [02:00<00:00, 36.3MB/s]
100% 5.96G/5.96G [02:00<00:00, 53.2MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 37.4MB/s]
100% 15.8M/15.8M [00:00<00:00, 29.0MB/s]

 98% 54.0M/54.9M [00:01<00:00, 37.5MB/s]
100% 54.9M/54.9M [00:01<00:00, 38.9MB/s]


In [ ]:
#Unzipping the Dataset
!unzip embeddings.zip
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  embeddings.zip
replace glove.840B.300d/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.840B.300d/glove.840B.300d.txt  a
A

  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [ ]:
#Libraries to be imported
import pandas as pd.              #DataFrame
import re                         #Regex library
from collections import Counter   #Module to help count iterable objects. Its a sub class dict datatype
import gensim                     #topic modelling, document indexing and similarity retrieval with large corpora.
import heapq                      #A datastructure which is maintained in ascending order. Whatever you insert, the ascending order is maintained.
from operator import itemgetter
from multiprocessing import Pool

In [ ]:
#Importing test dataset
test = pd.read_csv('/content/test.csv')
test.head(5)

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [ ]:
#Importing train dataset
train = pd.read_csv('/content/train.csv')
train.head(5)
#Insincere questions are labelled as 1.

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


## TEXT PREPROCESSING

In [ ]:
#Cleaning Special Characters and Removing Punctuations:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', ' ', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return 

In [ ]:
#Cleaning numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
#Removing Misspells:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', binary=True)
words = model.index2word

In [ ]:
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

#Finds all words
def words(text): return re.findall(r'\w+', text.lower())

*********************
START HERE
**********************
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

#All the vocab words are passed here
#More details on max function with key args.
#Here the vocab is passed with keywords i.e the words of our vocabulary. first we find a list of all possible 
#words from our 4 methods which could relate to our word. Then we check the probablity of each word by the function P and returm the max value.
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

#Let word be 'computer'. FUnction returns a list of matching words.
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

#If the word is there in the embeddings.
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

#word = 'BANANA'
#Returns all possible words with one edit of the actual word being passed.
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    
    #The splits contains tuple of different word combination.
    #LEN(WORD) = 6. i's range is 0 to 7. tuple = [(word[:0], word[0:]),(word[:1], word[1:])...(word[:6], word[6:]) 
    # split = [(, BANANA ),(B, ANANA),(BA, NANA),(BAN, ANA),(BANA, NA),(BANAN, A),(BANANA,)]
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    
    #In SPlits, for every combination where R exists(SInce that covers all our word combos; see list above) we delete
    #the first letter from R and join the word. Basically if we have added any letter in the word by mistake.
    deletes    = [L + R[1:]               for L, R in splits if R]

    #Combination where any two consecutive letters are exchanged. This is saved in transposes.
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

    #To check if any letter was incorrectly entered. We generate 26 such possibilities.
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]

    #To check if any letter was missing
    inserts    = [L + c + R               for L, R in splits for c in letters]
    
    #Returns a list of possible combinations of the word.
    return set(deletes + transposes + replaces + inserts)

#To check within the list of possible words, if there are more possible words. Nested loop.
#If 1 word had 5 possible words. then each of those 5 will have 5 more.
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#Building a vocabulary of words in the sentence.
def build_vocab(texts):
    #SPlitting the question into sentences list.
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

#Passsing the question to build vocab function.
vocab = build_vocab(train.question_text)
#vocab is a dictionary of all the question text with a count of each word.

#Creating a dictionary top_90k_words. We use the function nlargest from heapq library, where from the iterable object dict top 'n'/9000 rows are selected.
#The key determines by which column it is sorted. Here it is by the word count column.
top_90k_words = dict(heapq.nlargest(90000, vocab.items(), key=itemgetter(1)))

#We bring in multiple processing to improve effeciency.
pool = Pool(4)
#The correction method is called with all the top 90000 words being passed.
corrected_words = pool.map(correction,list(top_90k_words.keys()))

for word,corrected_word in zip(top_90k_words,corrected_words):
    if word!=corrected_word:
        print(word,":",corrected_word)

Streaming output truncated to the last 5000 lines.
INFPs? : NFPs
mammoth? : mammoth
soggy? : soggy
20L : 2L
plato? : plato
willing? : willing
dichlorine : dichloride
"Spirit : Spirit
guys) : guys
"did : did
choosing, : choosing
afford, : afford
Preferably, : Preferably
cheaper. : cheaper
Design, : Design
Presidents, : Presidents
desperate, : desperate
Naruto. : Naruto
keloid? : keloids
actor) : actor
Slim's : Slims
M-16 : -1
finished, : finished
instance) : instance
risky, : risky
Kapoor, : Kapoor
“Happy : Happy
news)? : news
Strike? : Strike
concept) : concept
Captain? : Captain
All" : All
United's : Uniteds
ketamine? : ketamine
IT/CS : ITCS
Rock" : Rock
Napoleon, : Napoleon
Olympus? : Olympus
Advanced) : Advanced
sari? : sari
"working : working
Israelis, : Israelis
NRIs? : NRIs
demise, : demise
fiancé? : fiancé
Giants? : Giants
colored? : colored
(criminal : criminal
y1 : 1
full-time, : fulltime
extrovert, : extrovert
80’s : 8s
city)? : city
constitutions? : constitutions
CCMT? : CCM

NameError: ignored